<a href="https://colab.research.google.com/github/kinjaljoshi/ma_rag_components/blob/master/api_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community

In [1]:
# Extended API object registry
api_registry = {
    "purchase_order": {
        "access_method": "api",
        "endpoint": "/api/purchase-orders",
        "params": ["po_id", "vendor_id"]
    },
    "sales_order": {
        "access_method": "api",
        "endpoint": "/api/sale-orders",
        "params": ["order_id"]
    }
}


In [6]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
import os

from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = api_key


llm = ChatOpenAI(temperature=0)

api_call_prompt = PromptTemplate.from_template("""
You are a smart assistant that helps generate Python function calls to invoke APIs.

Here is the available API registry with parameters:

{api_descriptions}

User query:
"{user_query}"

Generate a Python function call in this format:
call_api(<object_name>, <parameter_dict>)

Respond with **only the function call**.
""")

def format_api_descriptions(registry):
    lines = []
    for name, conf in registry.items():
        params = ", ".join(conf.get("params", []))
        lines.append(f"- {name}: endpoint={conf['endpoint']}, params=[{params}]")
    return "\n".join(lines)

api_chain = LLMChain(llm=llm, prompt=api_call_prompt)

def generate_api_function_call(user_query: str):
    descriptions = format_api_descriptions(api_registry)
    response = api_chain.run({
        "api_descriptions": descriptions,
        "user_query": user_query
    })
    return response.strip()
def call_api(object_name: str, params: dict):
    endpoint = api_registry[object_name]["endpoint"]
    print(f"Calling API: {endpoint}")
    print(f"With parameters: {params}")


In [7]:
query = "Get purchase orders for vendor V-1234"

func_call = generate_api_function_call(query)
print("LLM Function Call:", func_call)

exec(func_call)


<ipython-input-6-f82010c306f9>:40: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = api_chain.run({


LLM Function Call: call_api('purchase_order', {'vendor_id': 'V-1234'})
Calling API: /api/purchase-orders
With parameters: {'vendor_id': 'V-1234'}


In [8]:
query = "Get sales orders details for order X-9999"

func_call = generate_api_function_call(query)
print("LLM Function Call:", func_call)

exec(func_call)


LLM Function Call: call_api('sales_order', {'order_id': 'X-9999'})
Calling API: /api/sale-orders
With parameters: {'order_id': 'X-9999'}
